법정동의 경계 데이터(구역의 도형)    
https://business.juso.go.kr/addrlink/adresInfoProvd/guidance/provdAdresInfo.do   

법정동 데이터   
https://www.data.go.kr/data/15063424/fileData.do

두 데이터를 조인하여 내 데이터베이스에 저장     
socketUser 가 위치 정보를 전송하면 가장 가까운 지역을 찾기 위해서   

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
gdf = gpd.read_file('./11000/TL_SCCO_EMD.shp', encoding='euc-kr')
df = pd.read_csv('./법정동_20240208.csv', encoding='euc-kr')

In [3]:
# merge 하려면  key의 타입을 맞춰줘야 하고 key 이름이 같아야 함
gdf['EMD_CD'] = gdf['EMD_CD'].astype(int)
df = df.rename(columns={"법정동코드": "EMD_CD"})

In [4]:
gdf.head()

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110102,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.190 1..."
3,11110104,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110105,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1..."


In [5]:
# 현재 좌표계가 EPSG 5179 인데 이는 평면좌표계이므로 거리 등을 계산할 때 오차가 날 수 있다.
# 지구 곡면을 고려한 위도 경도 좌표로 바꿔줘야 정확한 거리 계산이 가능함.
# socketUser 핸드폰으로도 위도 경도 데이터를 받아오기 때문에 디비에도 위도 경도로 저장해 놓는 게 좋을 것 같다.

# shp 파일에는 좌표계 정보가 없으므로 명시적으로 넣어줌
gdf.crs = "EPSG:5179"

# 공간좌표계 컬럼 추가
gdf["geo_g"] = gdf["geometry"].to_crs(epsg=4326)
gdf['g_center'] = gdf['geo_g'].centroid
# 투영좌표계 중심점 추가
gdf["p_center"] = gdf["geometry"].centroid

/var/folders/6s/5sc4hlbx0vv8cz2rd1gwn_4w0000gn/T/ipykernel_3821/3635196011.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['g_center'] = gdf['geo_g'].centroid


In [104]:
gdf.head()

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry,geo_g,p_center,g_center
0,11110101,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1...","POLYGON ((126.97556 37.58968, 126.97549 37.589...",POINT (953149.185 1954555.436),POINT (126.96932 37.58921)
1,11110102,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1...","POLYGON ((126.97031 37.58418, 126.97033 37.584...",POINT (953024.188 1954034.783),POINT (126.96794 37.58451)
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.190 1...","POLYGON ((126.97400 37.58654, 126.97401 37.586...",POINT (953444.998 1954053.668),POINT (126.97271 37.58470)
3,11110104,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1...","POLYGON ((126.97356 37.58323, 126.97355 37.582...",POINT (953377.196 1953828.779),POINT (126.97195 37.58267)
4,11110105,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1...","POLYGON ((126.97353 37.58182, 126.97354 37.581...",POINT (953421.942 1953594.746),POINT (126.97247 37.58056)


In [105]:
# 법정동 데이터
df = df.sort_values(by="EMD_CD")
df.head()

,EMD_CD,시도명,시군구명,읍면동명,리명,순위,생성일자,삭제일자,과거법정동코드
20246,1100000000,서울특별시,NaN,NaN,NaN,11.0,1988-04-23,NaN,NaN
20247,1111000000,서울특별시,종로구,NaN,NaN,1.0,1988-04-23,NaN,NaN
20248,1111010100,서울특별시,종로구,청운동,NaN,1.0,1988-04-23,NaN,NaN
20249,1111010200,서울특별시,종로구,신교동,NaN,2.0,1988-04-23,NaN,NaN
20250,1111010300,서울특별시,종로구,궁정동,NaN,3.0,1988-04-23,NaN,NaN


In [106]:
# 지리 데이터는 법정동 코드가 11110101 인데
# 일반 데이터는            1111010100
# 지리 데이터는 읍면동까지고 리 에 대한 코드는 빠져있기 때문이다.
# merge 시 key 를 맞춰주기 위해 지리데이터에 00 삽입
gdf['EMD_CD'] = gdf['EMD_CD'] * 100

In [107]:
merged = pd.merge(df, gdf, on='EMD_CD', how='inner')

In [108]:
merged.head()

,EMD_CD,시도명,시군구명,읍면동명,리명,순위,생성일자,삭제일자,과거법정동코드,EMD_ENG_NM,EMD_KOR_NM,geometry,geo_g,p_center,g_center
0,1111010100,서울특별시,종로구,청운동,NaN,1.0,1988-04-23,NaN,NaN,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1...","POLYGON ((126.97556 37.58968, 126.97549 37.589...",POINT (953149.185 1954555.436),POINT (126.96932 37.58921)
1,1111010200,서울특별시,종로구,신교동,NaN,2.0,1988-04-23,NaN,NaN,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1...","POLYGON ((126.97031 37.58418, 126.97033 37.584...",POINT (953024.188 1954034.783),POINT (126.96794 37.58451)
2,1111010300,서울특별시,종로구,궁정동,NaN,3.0,1988-04-23,NaN,NaN,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.190 1...","POLYGON ((126.97400 37.58654, 126.97401 37.586...",POINT (953444.998 1954053.668),POINT (126.97271 37.58470)
3,1111010400,서울특별시,종로구,효자동,NaN,4.0,1988-04-23,NaN,NaN,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1...","POLYGON ((126.97356 37.58323, 126.97355 37.582...",POINT (953377.196 1953828.779),POINT (126.97195 37.58267)
4,1111010500,서울특별시,종로구,창성동,NaN,5.0,1988-04-23,NaN,NaN,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1...","POLYGON ((126.97353 37.58182, 126.97354 37.581...",POINT (953421.942 1953594.746),POINT (126.97247 37.58056)


In [109]:
merged = merged.rename(columns={'EMD_CD': 'code', '시도명': 'c1', '시군구명': 'c2', '읍면동명': 'c3', '리명': 'c4', '순위': 'order', '생성일자': 'created_at', '삭제일자': 'deleted_at', '과거법정동코드': 'prev_code', 'EMD_ENG_NM': 'eng_name', 'EMD_KOR_NM': 'name'})

In [110]:
# merged['geo_g'] = merged['geo_g'].apply(lambda geom: geom.wkt)
# merged["p_center"] = merged["p_center"].apply(lambda geom: geom.wkt)

In [111]:
merged.to_csv('seoul_emd_geometry.csv', sep='|')